In [35]:
from azureml.core import Workspace 

In [37]:
ws = Workspace.create(name = 'my-ml-workspace',
                      subscription_id = 'b1e9d77a-82fa-47dc-addb-d917728f82c4',
                      resource_group = 'resourcegroup_auto_ml',
                      location = 'eastus')
                      

                      
                      
                      

Deploying StorageAccount with name mymlworkstorage0fb91699d.
Deploying AppInsights with name mymlworkinsights6fcddc29.
Deployed AppInsights with name mymlworkinsights6fcddc29. Took 1.79 seconds.
Deploying KeyVault with name mymlworkkeyvault478f923c.
Deployed KeyVault with name mymlworkkeyvault478f923c. Took 16.92 seconds.
Deployed StorageAccount with name mymlworkstorage0fb91699d. Took 21.97 seconds.
Deploying Workspace with name my-ml-workspace.
Deployed Workspace with name my-ml-workspace. Took 18.98 seconds.


In [38]:
from azureml.core import Datastore

az_store = Datastore.register_azure_blob_container(workspace = ws,
                                                   datastore_name ='azure_datastore',
                                                   account_name = 'mymlworkstorage0fb91699d',
                                                   container_name = 'myblobcontainer',
                                                   account_key = 'A4RVcGEl2iSTWyHij5U8fzHhDWk7HLr2Q+KvAoVVOzGzO1TY4bxj2tk6/iPypCxCC0480o6Yyqow+AStoHorAQ==')

                                        
                                                   

In [39]:
from azureml.core import Dataset

csv_path = [(az_store, 'data/defaults.csv')]

defaults_dataset = Dataset.Tabular.from_delimited_files(path = csv_path)

input_ds = defaults_dataset.register(workspace = ws,
                                         name = 'Defaults',
                                         create_new_version = True)



In [46]:
csv_path = [(az_store, 'data/adultincome+trunc (1).csv')]

defaults_dataset = Dataset.Tabular.from_delimited_files(path = csv_path)

input_ds = defaults_dataset.register(workspace = ws,
                                         name = 'AdultIncome',
                                         create_new_version = True)


In [57]:
from azureml.core import Environment 
from azureml.core.environment import CondaDependencies

myenv = Environment(name= 'MyEnviroment')

myenv_dep = CondaDependencies.create(conda_packages=['scikit-Learn','pip', 'pandas'],
                                                     pip_packages=['azureml-defaults'])
myenv.python.conda_dependencies = myenv_dep 



In [58]:
print('Registering the enviroment..')

myenv.register(ws)



Registering the enviroment..


{
    "assetId": "azureml://locations/eastus/workspaces/542c14ab-fc66-4336-b1a3-10d030ebe07e/environments/MyEnviroment/versions/2",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnviroment",
    "pytho

In [42]:
from azureml.core.compute import AmlCompute

cluster_name = 'my-cluster-001'

if cluster_name not in ws.compute_targets:
    print("creating compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(vm_size ='STANDARD_D11_V2',
                                                       max_nodes =2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluser = ws.compute_target[cluster_name]
    print(cluster_name, "compute cluster found")
          
          

creating compute cluster with name:  my-cluster-001
Provisioning operation finished, operation "Succeeded"


In [43]:
from azureml.core import ScriptRunConfig 

script_config = ScriptRunConfig(source_directory ='.',
                                script = 'hyperdrive.py',
                                environment = myenv,
                                compute_target= cluster)


In [25]:
!pip install azureml-train
from azureml.train.hyperdrive import GridParameterSampling, choice

hyper_param = GridParameterSampling(
                                   {'--n_estimator': choice(10,20,50,100),
                                    '--min_sample_leaf': choice(1,2,5)
                                   })


In [28]:
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal

hyper_config = HyperDriveConfig(run_config = script_config,
                               hyperparameter_sampling = hyper_param,
                               policy = None,
                               primary_metric_name = 'accuracy',
                               primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                               max_total_runs=20,
                               max_concurrent_runs=2)

In [31]:
from azureml.core import Experiment, Run

new_experiment = Experiment(workspace =ws, name='Hyperdrive_exp001')

new_run = new_experiment.submit(config=hyper_config)

new_run.wait_for_completion(show_output = True)

RunId: HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b
Web View: https://ml.azure.com/runs/HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b?wsid=/subscriptions/b1e9d77a-82fa-47dc-addb-d917728f82c4/resourcegroups/resourcegroup_auto_ml/workspaces/my-ml-workspace&tid=704d822c-358a-4784-9a16-49e20b75f941

Streaming azureml-logs/hyperdrive.txt

[2022-08-17T20:48:41.130698][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2022-08-17T20:48:41.9556345Z][SCHEDULER][INFO]Scheduling job, id='HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b_0' 
[2022-08-17T20:48:42.052446][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2022-08-17T20:48:42.1162812Z][SCHEDULER][INFO]Scheduling job, id='HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b_1' 
[2022-08-17T20:48:42.2781823Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b_0' 
[2022-08-17T20:48:42.3152264Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_0844e973-bc8d-

{'runId': 'HD_0844e973-bc8d-45ae-a3a4-32b19dc88c5b',
 'target': 'my-cluster-001',
 'status': 'Canceled',
 'startTimeUtc': '2022-08-17T20:48:40.432694Z',
 'endTimeUtc': '2022-08-17T20:51:40.373815Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7f2efa25-b15b-405f-9faa-2c1e36178468',
  'user_agent': 'python/3.9.7 (macOS-10.16-x86_64-i386-64bit) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '12'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',


In [48]:
from azureml.core.compute import AksCompute 

cluster_name = 'aks-cluster-001'

aks_config = AksCompute.provisioning_configuration(location = 'eastus',
                                                   vm_size = 'STANDARD_D11_V2',
                                                   agent_count = 1,
                                                   cluster_purpose = 'DevTest')

from azureml.core.compute import ComputeTarget

production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)
                                                   
                                                   
                                                   

In [60]:
from azureml.core.model import InferenceConfig 

inference_config = InferenceConfig(environment = myenv, 
                                   entry_script = 'scoring_script.py',
                                   source_directory = './data')

from azureml.core.webservice import AksWebservice 

deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                  memory_gb =1 )


In [61]:
from azureml.core.model import Model 

model = ws.models['AdultIncome']
service = Model.deploy(workspace = ws,
                      name = 'adultincome-service',
                      models = [model],
                      inference_config = inference_config,
                      deployment_config = deploy_config,
                      deployment_target = production_cluster)

service.wait_for_deployment(show_output=True)




Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-18 15:40:04-04:00 Creating Container Registry if not exists.
2022-08-18 15:40:04-04:00 Registering the environment.
2022-08-18 15:40:04-04:00 Building image..
2022-08-18 15:50:21-04:00 Creating resources in AKS..
2022-08-18 15:50:22-04:00 Submitting deployment to compute.
2022-08-18 15:50:22-04:00 Checking the status of deployment adultincome-service..
2022-08-18 15:51:12-04:00 Checking the status of inference endpoint adultincome-service.
Succeeded
AKS service creation operation finished, operation "Succeeded"


'/bin/bash: /azureml-envs/azureml_ac08a2104f0fdc8800bbe84fc0c5fe69/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_ac08a2104f0fdc8800bbe84fc0c5fe69/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_ac08a2104f0fdc8800bbe84fc0c5fe69/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_ac08a2104f0fdc8800bbe84fc0c5fe69/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2022-08-18T19:33:52,355056196+00:00 - gunicorn/run \n2022-08-18T19:33:52,355908405+00:00 - iot-server/run \n2022-08-18T19:33:52,357846325+00:00 - rsyslog/run \nbash: /azureml-envs/azureml_ac08a2104f0fdc8800bbe84fc0c5fe69/lib/libtinfo.so.6: no version information available (required by bash)\n2022-08-18T19:33:52,364252993+00:00 - nginx/run \n2022-08-18T19:33:52,369866552+00:00 | gunicorn/run | \n2022-08-18T19:33:52,380823367